# A gentle introduction

`map_blocks` is inspired by the `dask.array` function of the same name and lets
you map a function on blocks of the xarray object (including Datasets!).

At _compute_ time, your function will receive an xarray object with concrete
(computed) values along with appropriate metadata. This function should return
an xarray object.


## Setup

In [ ]:
import dask
import numpy as np
import xarray as xr

First lets set up a `LocalCluster` using [dask.distributed](https://distributed.dask.org/).

You can use any kind of dask cluster. This step is completely independent of
xarray. While not strictly necessary, the dashboard provides a nice learning
tool.


In [ ]:
from dask.distributed import Client

client = Client()
client

<p>&#128070</p> Click the Dashboard link above. Or click the "Search" button in the dashboard.

Let's test that the dashboard is working..


In [ ]:
import dask.array

dask.array.ones((1000, 4), chunks=(2, 1)).compute()  # should see activity in dashboard

Let's open a dataset. We specify `chunks` so that we create a dask arrays for the DataArrays

In [ ]:
ds = xr.tutorial.open_dataset("air_temperature", chunks={"time": 100})
ds

## Simple example

Here is an example

In [ ]:
def time_mean(obj):
    # use xarray's convenient API here
    # you could convert to a pandas dataframe and use pandas' extensive API
    # or use .plot() and plt.savefig to save visualizations to disk in parallel.
    return obj.mean("lat")


ds.map_blocks(time_mean)  # this is lazy!

In [ ]:
# this will calculate values and will return True if the computation works as expected
ds.map_blocks(time_mean).identical(ds.mean("lat"))

### Exercise

Try applying the following function with `map_blocks`. Specify `scale` as an
argument and `offset` as a kwarg.

The docstring should help:
https://docs.xarray.dev/en/stable/generated/xarray.map_blocks.html

```
def time_mean_scaled(obj, scale, offset):
    return obj.mean("lat") * scale + offset
```


### More advanced functions

`map_blocks` needs to know what the returned object looks like _exactly_. It
does so by passing a 0-shaped xarray object to the function and examining the
result. This approach cannot work in all cases For such advanced use cases,
`map_blocks` allows a `template` kwarg. See
https://docs.xarray.dev/en/stable/user-guide/dask.html#map-blocks for more details


In [ ]:
client.close()